# *FEniCS tutorial:* Heat equation with Dirichlet boundary conditions

In this demo, we solve the two-dimensional diffusion equation with Dirichlet
boundary conditions $u_D$ and source term $f$. Both are chosen so as to yield
an exact analytic result against which we can compare our numerical results.

$$
\begin{align}
  u'   &= \nabla^2 u + f     \quad\text{in the unit square} \\
  u    &= u_D  \hphantom{u+f}\quad\text{on the boundary} \\
  u    &= u_0  \hphantom{u+f}\quad\;\text{at $t = 0$} \\
  u_D  &= 1 + x^2 + \alpha y^2 + \beta t \\
  f    &= \beta - 2 (1 + \alpha)
\end{align}
$$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time as tm

# import os
# import re
# from color import *
# from mplotg import *
# from IPython.display import Image
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"# 

In [ ]:
# from mplotg import *
# import matplotlib.pyplot as plt

def multiplot_from_generator(g, num_columns, figsize_for_one_row=None):
    # call 'next(g)' to get past the first 'yield'
#     next(g)
    # default to 15-inch rows, with square subplots
    if figsize_for_one_row is None:
        figsize_for_one_row = (15, 15/num_columns)
    try:
        while True:
            # call plt.figure once per row
            plt.figure(figsize=figsize_for_one_row)
            for col in range(num_columns):
                ax = plt.subplot(1, num_columns, col+1)
                next(g)
    except StopIteration:
        pass

In [ ]:
from fenics import *

In [ ]:
T = 2.0            # final time
num_steps = 10     # number of time steps
dt = T / num_steps # time step size
alpha = 3          # parameter alpha
beta = 1.2         # parameter beta

In [ ]:
# Create mesh and define function space
nx = ny = 8
mesh = UnitSquareMesh(nx, ny)
V = FunctionSpace(mesh, 'P', 1)

In [ ]:
# Define boundary condition
u_D = Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t',
                 degree=2, alpha=alpha, beta=beta, t=0)

def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, u_D, boundary)

In [ ]:
# Define initial value
u_n = interpolate(u_D, V)
#u_n = project(u_D, V)

In [ ]:
# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(beta - 2 - 2*alpha)
F = u*v*dx + dt*dot(grad(u), grad(v))*dx - (u_n + dt*f)*v*dx
a, L = lhs(F), rhs(F)

In [ ]:
# Time-stepping
u = Function(V)
t = 0
for n in range(num_steps):

    # Update current time
    t += dt
    u_D.t = t

    # Compute solution
    solve(a == L, u, bc)

    # Plot solution
    plot(u)

    # Compute error at vertices
    u_e = interpolate(u_D, V)
    error = np.abs(u_e.vector() - u.vector()).max()
    print('t = %.2f: error = %.3g' % (t, error))

    # Update previous solution
    u_n.assign(u)

In [ ]:
def evolve():
    # Time-stepping
    u = Function(V)
    t=0
    for n in range(num_steps):
    
        # Update current time
        t += dt
        u_D.t = t

        # Compute solution
        solve(a == L, u, bc)

        # Plot solution
        yield plot(u)

        # Compute error at vertices
        u_e = interpolate(u_D, V)
        # error = np.abs(u_e.vector().array() - u.vector().array()).max()
        error = np.abs(u_e.vector() - u.vector()).max()
        print('t = %.2f: error = %.3g' % (t, error))

        # Update previous solution
        u_n.assign(u)

In [ ]:
multiplot_from_generator(evolve(), 5, (15, 3))